**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import operator
import functools
import itertools

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KDTree
%load_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [273]:
#%%pycodestyle


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):
        if self.algorithm == 'brute':
            self.fit_brute(X, y)
        elif self.algorithm == 'kd_tree':
            self.fit_kd_tree(X, y)

    def fit_brute(self, X, y):
        self.X = np.copy(X)
        self.y = np.copy(y)
        self.classes = np.unique(y)

        def what_class(tmp_class):
            res = [i for i in range(len(self.classes))
                   if self.classes[i] == tmp_class]
            return res[0]
        self.y = np.asarray(list(map(what_class, self.y)))

    def fit_kd_tree(self, X, y):
        self.tree = KDTree(X)
        self.X = np.copy(X)
        self.y = np.copy(y)
        self.classes = np.unique(y)

        def what_class(tmp_class):
            res = [i for i in range(len(self.classes))
                   if self.classes[i] == tmp_class]
            return res[0]
        self.y = np.asarray(list(map(what_class, self.y)))

    def dist(x, y):
        return np.linalg.norm(x-y)

    def predict(self, X):
        if self.algorithm == 'brute':
            return self.predict_brute(X)
        elif self.algorithm == 'kd_tree':
            return self.predict_kd_tree(X)

    def predict_brute(self, X):
        def predict_one_brute(x):
            dist = np.argsort(np.linalg.norm(self.X-x, axis=1))
            return self.classes[np.argsort(np.bincount(
                self.y[dist[0:self.n_neighbors]],
                minlength=len(self.classes)))[-1]]
        y = np.asarray(list(map(predict_one_brute, X)))
        return y

    def predict_kd_tree(self, X):
        def predict_one_kd_tree(x):
            ind = np.ravel(self.tree.query(x.reshape(1, x.shape[0]),
                                           k=self.n_neighbors,
                                           return_distance=False))
            return self.classes[np.argsort(np.bincount(
                self.y[ind],
                minlength=len(self.classes)))[-1]]
        y = np.asarray(list(map(predict_one_kd_tree, X)))
        return y

    def predict_proba(self, X):
        if self.algorithm == 'brute':
            return self.predict_proba_brute(X)
        elif self.algorithm == 'kd_tree':
            return self.predict_proba_kd_tree(X)

    def predict_proba_brute(self, X):
        def predict_proba_one_brute(x):
            dist = np.argsort(np.linalg.norm(self.X-x, axis=1))
            return np.bincount(self.y[dist[0:self.n_neighbors]],
                               minlength=len(self.classes)) / self.n_neighbors
        y = np.asarray(list(map(predict_proba_one_brute, X)))
        return y

    def predict_proba_kd_tree(self, X):
        def predict_proba_one_kd_tree(x):
            ind = np.ravel(self.tree.query(x.reshape(1, x.shape[0]),
                                           k=self.n_neighbors,
                                           return_distance=False))
            return np.bincount(self.y[ind],
                               minlength=len(self.classes)) / self.n_neighbors
        y = np.asarray(list(map(predict_proba_one_kd_tree, X)))
        return y

    def score(self, X, y):
        y1 = self.predict(X)
        y1 = np.equal(y1, y)
        z = np.sum(y1) / y.size
        return z

98:1: W391 blank line at end of file


**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [238]:
iris = datasets.load_iris()

In [239]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [240]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [241]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [242]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [243]:
%time clf.fit(X_train, y_train)

CPU times: user 691 µs, sys: 97 µs, total: 788 µs
Wall time: 668 µs


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [244]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.18 ms, sys: 0 ns, total: 1.18 ms
Wall time: 1.06 ms


In [245]:
%time clf.predict(X_test)

CPU times: user 0 ns, sys: 2.29 ms, total: 2.29 ms
Wall time: 1.8 ms


array([0, 1, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 1, 0, 1])

In [246]:
%time my_clf.predict(X_test)

CPU times: user 2.76 ms, sys: 0 ns, total: 2.76 ms
Wall time: 3.24 ms


array([0, 1, 0, 2, 1, 0, 0, 2, 2, 1, 2, 2, 1, 0, 1])

In [247]:
%time clf.predict_proba(X_test)

CPU times: user 2.02 ms, sys: 282 µs, total: 2.3 ms
Wall time: 1.71 ms


array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [248]:
%time my_clf.predict_proba(X_test)

CPU times: user 2.38 ms, sys: 333 µs, total: 2.71 ms
Wall time: 1.88 ms


array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [258]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [259]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [260]:
%time clf.fit(X_train, y_train)

CPU times: user 1.5 ms, sys: 211 µs, total: 1.71 ms
Wall time: 1.19 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [261]:
%time my_clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 1.41 ms, total: 1.41 ms
Wall time: 1.1 ms


In [262]:
%time clf.predict(X_test)

CPU times: user 1.97 ms, sys: 278 µs, total: 2.24 ms
Wall time: 3.16 ms


array([2, 0, 1, 1, 1, 0, 2, 0, 0, 1, 2, 1, 1, 2, 0])

In [263]:
%time my_clf.predict(X_test)

CPU times: user 3.26 ms, sys: 463 µs, total: 3.72 ms
Wall time: 4.58 ms


array([2, 0, 1, 1, 1, 0, 2, 0, 0, 1, 2, 1, 1, 2, 0])

In [264]:
%time clf.predict_proba(X_test)

CPU times: user 1.46 ms, sys: 0 ns, total: 1.46 ms
Wall time: 1.28 ms


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [265]:
%time my_clf.predict_proba(X_test)

CPU times: user 0 ns, sys: 2.73 ms, total: 2.73 ms
Wall time: 2.28 ms


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [266]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [2]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [4]:
data = newsgroups['data']
target = newsgroups['target']

In [5]:
target

array([7, 4, 4, ..., 3, 1, 8])

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [7]:
data_tok = []
for string in data:
    data_tok.append(string.lower())
data_tok#realize here
# data_tok should be a list of lists of tokens for each line in data.

['i was wondering if anyone out there could enlighten me on this car i saw\nthe other day. it was a 2-door sports car, looked to be from the late 60s/\nearly 70s. it was called a bricklin. the doors were really small. in addition,\nthe front bumper was separate from the rest of the body. this is \nall i know. if anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.',
 "a fair number of brave souls who upgraded their si clock oscillator have\nshared their experiences for this poll. please send a brief message detailing\nyour experiences with the procedure. top speed attained, cpu rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\ni will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't an

In [ ]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.